# Notes
## Questions
### General
* How to secure your API_KEY when you're doing all this URL parsing with your `key` embedded? Suggestions?
* Do we need to worry about overwriting variables that use same names? E.g., `params`, `base_endpoint`, etc. that we used again and again for the different Places endpoints? Found myself wanting to have unique names for each but just curious.
* What are your thoughts about building an REST vs. GraphQL API? Seems like GQL could maybe save some money in the long term ...actually, take that back because the pricing is based on the actual `data fields` you request. Perhaps GQL could allow you to be more specific and therefore save you some money instead of getting *all* fields returned.
### Building the API Client Class
* What about the `base_endpoint` URL? Could/should this be added as a class variable? Or should it be added within the `__init__` method? When do you choose one or the other? Why not just put all of these inside the `__init__` function? Just seems redundant somehow...
* To make our Client more robust, should we add `assert` or `raise Exception` in some places? Is that common for `class` objects?
* Do you like to use constants? `API_KEY` vs. `api_key`?
* Better understanding `location` and the `extract_latlng()` method. Okay, so when we init a new instance, we're setting `self.location_query = address_or_postal_code`. Even if an address is passed during the initialization, we're going to set the `self.location_query` and `self.lat`, `self.lng` values. **However**, if we *do not* pass an `address_or_postal_code` during initialization, then the `self.lat` and `self.lng` values will be their defaults `None`. Then later on a user may also choose to update the address/location via the `search()` method. We're giving them the option to pass a new `location` argument to `search(self, location)` method. If a new `location` is provided, then we re-run the `extract_latlng()` method but now passing it this updated `location` value. However, before we do any of that, we first (within the `search()` method) set some *local* function vars `lat, lng = self.lat, self.lng` since we want to first get whatever those instance values are currently before we run the `extract_latlng()` method to get updated values. This means that if a `location` has indeed been passed to `extract_latlng()` (any time *after* the instance has been initialized), then we're going to update the `self.location_query`, `self.lat`, and `self.lng` class variable values.
**It seems to be a difference of referencing the `class` variable value (`self.lat`) vs. the `local` variable value (`lat`).** Recall that originally we have `params["address"]: self.location_query`, which during initialization could be `None` (if `address_or_postal_code is None`). 
* What's the difference between `self.lat = lat` vs. `lat = self.lat`? 
## Setup - Geocoding & Places API w/ Google Maps
* There's actually a [Python Client for Google Maps Services](https://github.com/googlemaps/google-maps-services-python) already available but we're going to build our own custom one for more practice.
* Lots of APIs/SDKs available in the API Library. Geolocation is for devices. Embedded Maps JS for displaying the map in a website, etc. **You find your APIs you want to enable and then click `enable`**
* `Geocoding API` to convert between addresses and geographic coordinates. `Places API` 
* `APIs & Services` > `Credentials` > `API Keys` to create our API Keys (`OAuth2` requires me to have a web application/callback place). We're going to use our `API key` to make requests and we'll handle the security soon.
* **Restrict** the API key by HTTP referrers (my website), a specific IP address (i.e., my home), Android/iOS apps, etc. You can also restrict which APIs this API key has access to (i.e., Places and Geocoding).
## Exploring the APIs
### Geocoding API
* In the tutorial there's a `Client` header. I need to wrap my head around this term within this context. We built an API Client to use Spotify's API. I believe we're doing the same but for Google.
* Like most APIs you have an `endpoint` URL, we pass in `parameters` and `api_key`, and execute the `request`.
* [Docs for Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview) with a sample `request` template: `https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,
+Mountain+View,+CA&key=YOUR_API_KEY` --- `&` represents a new parameter in URLs so we know it's a JSON/`dict` like structure.
* If I use `d['key']` I'll get a `KeyError`. If I use `d.get('key')` I'll get `None` returned.
* To *reverse engineer* an long url string with the query inside, we're going to use `urllib.parse` library and its `urlparse` and `parse_qsl` methods. You can do `urlparse(url_to_parse)` to get a `ParseResult` object that has a bunch of components (scheme, netloc, path, params, query, etc.). The goal is to reverse engineer a long url string into a functional `endpoint` we could use again: `endpoint: str = f"{base_endpoint}?{urlencode(query_dict)}"`
* There's a lot more that can be done. For example, say you have a user input form where they enter in their address. You could extract their input, send a request to this Geocoding API, get the `formatted_address` from the results, and display this back to the user to have them confirm the correct spelling/address.
### Places API
* We're using the [Place Search](https://developers.google.com/places/web-service/search) function but there is also Place Details, Types, Data Fields, etc.
* An example Place request: `https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters` **Note: This base Place request endpoint only returns *one* result! We have to use the nearby search to get more results.**
Likes to hardcode 'output' to json instead in the base endpoint.
key, input and inputtype are required parameters
Inside the params we send in the endpoint url, we can add some optional parameters such as fields and locationbias. The fields allows us to specify the types of place data to return (seems pretty flexible like GraphQL in a way). Within locationbias we can get results in a specified area, which can leverage the longitude and latitude latlng data we got earlier from Geocoding API.
locationbias has has additional options where the final str could look like: ipbias, point:lat,lng, circle:radius@lat,lng, rectangle:south,west|north,east. (%3A is byte string for : I believe).
fields can be something like fields=address_component,name,geometry,rating,opening_hours,. $$$ Be careful! Certain fields cost more to extract!
To get more than one result, we must use the Nearby Search requests. NOTE that this request will return all data fields available so you will be billed accordingly! This is where GraphQL could be helpful probably. The Nearby Search has a slightly different endoint to construct with key, location and radius as required parameters: https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
Once we find a place based on our type of search (request), we'll get a Place Details request that has more information that we can use: https://maps.googleapis.com/maps/api/place/details/json?place_id=ChIJN1t_tDeuEmsRUsoyG83frY4&fields=name,rating,formatted_phone_number&key=YOUR_API_KEY
## Building the Google Maps API Client (or, Client API) Class
### Geocoding (`extract_latlng()`)
* First thing we need is the extracted `lat` and `lng` values
* Move `data_type='json'` as a class variable so every request we make uses `json`.
* Same with the `address` (or location). That's going to come with every request so can just pull it out and make it into a class variable, e.g., `location_query: t.Optional[str] = None`
* We can set `lat` and `lng` values back to the class itself using `self.lat = lat` and `self.lng = lng`. Otherwise, they'll remain as `None` as per their defaults. This allows us to use the `self.lat` and `self.lng` values later within other class methods, etc. 
* In fact, after these changes we could actually initialize this Client (class) this way. Meaning we can define the `__init__` function and have it require an `address_or_postal_code` argument, which can then be used to extract the `lat` and `lng` values by doing `self.extract_latlng()` as part of the instance initialization. We can have `address_or_postal_code` be an *optional* argument by defaulting to `None`. This means that the user can just create a new instance of the Client without having to also specify an `address` from the beginning. However, if the user does provide/pass a `address_or_postal_code`, then we could handle/update the `lat` and `lng` values based on the provided location.
* Same goes for `api_key` as we must have one. Therefore let's add it as a class variable and default to `None`. Then let's `raise Exception` if one isn't passed as an argument.
### Places Nearby Search (`search()`)
* Going to just implement the nearby searches feature instead of just single `findplacefromtext`.
* We could consider giving the user the chance to change the `lat` and `lng` values when using the `search()` class method we're building. To do this, we can add a `location=None` parameter to our `search(self, keyword="Tex-Mex food", location=None)` definition. So if a `location` is passed, then we'd have to update the `lat`, `lng` values using our `extract_latlng()` method. However, we want to reset/update the values based on `location`, so it makes sense to add `location=None` to the `extract_latlng(self, location=None)` method as well.
    * ??         # NOTE ?? - Why not just set self.location_query = location?
### Places Detail (`detail()`)
* More straight-forward since we don't mess with `location', only `place_id` and `fields`.
## Challenge
* Improve the way we construct our `endpoint` URLs.
* Refactor the way we make our `request` as we repeat a lot of code. A good Client would do it once and change it for you depending on the type of request or endpoint you're trying to reach.
* Implement ways to enrich a `Pandas` dataset by pulling in some of this Google Maps data.

In [62]:
import typing as t
from urllib.parse import urlencode, urlparse, parse_qsl

import requests

from google_geocoding_places_api import settings

In [63]:
API_KEY = settings.API_KEY

## Geocoding API

In [106]:
# Setup a request: https://maps.googleapis.com/maps/api/geocode/outputFormat?parameters
data_type: str = "json"  # outputFormat
params: t.Dict = {"address": "1600 Amphitheatre Parkway, Mountain View, CA", "key": API_KEY}
url_params: str = urlencode(params)
endpoint: str = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
url: str = f"{endpoint}?{url_params}"

In [65]:
# We have our 'url' ready for the request but we want to return the latitude and longitude
# Let's convert this into a function
def extract_latlng(address: str, data_type: str = "json") -> t.Tuple:
      # outputFormat
    params: t.Dict = {"address": address, "key": API_KEY}
    url_params: str = urlencode(params)
    endpoint: str = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
    url: str = f"{endpoint}?{url_params}"

    # Now we have our request url let's make the request
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        print(f"Request failed! {url}")
        return {}
    
    # Try to set/extract the lat lng data from 'location' key 
    # r.json()['results'][0]['geometry']['location']
    # {'lat': 37.4220578, 'lng': -122.0840897}
    latlng: t.Dict[str, float] = {}
    try:
        latlng = r.json()['results'][0]['geometry']['location']
        # latlng['lat'] = r.json()['results'][0]['geometry']['location']['jjkk']
        # latlng['lng'] = r.json()['results'][0]['geometry']['location']['lng']
    except:
        pass

    # Use .get('key') to get result or None. If I use d['key'] could get KetError
    return latlng.get('lat'), latlng.get('lng')  # (None, None)
    

In [58]:
extract_latlng("1600 Amphitheatre Parkway, Mountain View, CA")

(37.4220578, -122.0840897)

In [86]:
# Reverse engineer the url string with urlparse and parse_sql
# from urllib.parse import urlparse, parse_sql
url_to_parse: str = f"https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway%2C+Mountain+View%2C+CA&key={API_KEY}"

In [107]:
parsed_result = urlparse(url_to_parse) # ParseResult object
# print(parsed_result)

In [108]:
query_string: str = parsed_result.query
# parse_qsl(query_string)
# print(type(parse_qsl(query_string)))  # List
# print(type(parse_qsl(query_string)[0]))  # Tuple
# address_param: str = parse_qsl(query_string)[0][1]
# key_param: str = parse_qsl(query_string)[1][1]
# print(address_param)
# print(key_param)

In [109]:
# Convert this list into a dict. This is replicating our url_params Dict above basically
query_dict: t.Dict[str, str] = dict(parse_qsl(query_string))
# query_dict


In [110]:
# Let's turn all of this into a functional endpoint
base_endpoint: str = f"{parsed_result.scheme}://{parsed_result.netloc}{parsed_result.path}"
# print(base_endpoint)  # https://maps.googleapis.com/maps/api/geocode/json
endpoint: str = f"{base_endpoint}?{urlencode(query_dict)}"
# endpoint

## Places API

In [111]:
# https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters
# Let's build our base endpoint
lat, lng = (37.4220578, -122.0840897)  # Mountain View for testing only
places_base_endpoint: str = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
params: t.Dict = {
    "key": API_KEY,
    "input": "Tex-Mex food",
    "inputtype": "textquery",  # or, 'phonenumber'
    "fields": "formatted_address,name,geometry,place_id"
}
# Optional parameters like fields and locationbias can take in latlng coordinates
locationbias: str = f"point:{lat},{lng}"
# Check if circular option is used
use_circular: bool = True
if use_circular:
    radius: int = 1000  # meters
    locationbias = f"circle:{radius}@{lat},{lng}"

# Add locationbias key: value to params
params['locationbias'] = locationbias

params_encoded: str = urlencode(params)
places_endpoint: str = f"{places_base_endpoint}?{params_encoded}"
# print(places_endpoint)

# Let's now try our request with our above details
r = requests.get(places_endpoint)
r.json()


{'candidates': [{'formatted_address': '2560 W El Camino Real, Mountain View, CA 94040, United States',
   'geometry': {'location': {'lat': 37.4008824, 'lng': -122.111055},
    'viewport': {'northeast': {'lat': 37.40223172989272,
      'lng': -122.1097056201073},
     'southwest': {'lat': 37.39953207010728, 'lng': -122.1124052798927}}},
   'name': "Chili's Grill & Bar",
   'place_id': 'ChIJ8ZTVZJmwj4ARQFv0RXspg3A'}],
 'status': 'OK'}

In [112]:
# Let's try the Nearby Search request to get multiple results
# https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
places_base_nearby_endpoint: str = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_nearby: t.Dict = {
    "key": API_KEY,
    "location": f"{lat},{lng}",
    "radius": 1500,  # meters
    "keyword": "Tex-Mex food",

}
params_nearby_encoded: str = urlencode(params_nearby)
places_nearby_endpoint: str = f"{places_base_nearby_endpoint}?{params_nearby_encoded}"

r_nearby = requests.get(places_nearby_endpoint)
results: t.List[dict] = r_nearby.json()['results']
# print(results)



In [113]:
# Once we get our place either from Place or Nearby request we can
# use Place Detail along with place_id for a detail_lookup_request
# https://maps.googleapis.com/maps/api/place/details/json?place_id=ChIJN1t_tDeuEmsRUsoyG83frY4&fields=name,rating,formatted_phone_number&key=YOUR_API_KEY
place_id: str = "ChIJ8ZTVZJmwj4ARQFv0RXspg3A" # Chili's - Or, Australia - "ChIJN1t_tDeuEmsRUsoyG83frY4"  
places_base_details_endpoint: str = "https://maps.googleapis.com/maps/api/place/details/json"
params_details = {
    "place_id": f"{place_id}",
    "fields": "name,rating,formatted_phone_number,formatted_address",
    "key": API_KEY
}
params_details_encoded: str = urlencode(params_details)
places_details_endpoint: str = f"{places_base_details_endpoint}?{params_details_encoded}"

r_details = requests.get(places_details_endpoint)
r_details.json()


{'html_attributions': [],
 'result': {'formatted_address': '2560 W El Camino Real, Mountain View, CA 94040, USA',
  'formatted_phone_number': '(650) 941-2227',
  'name': "Chili's Grill & Bar",
  'rating': 4.1},
 'status': 'OK'}